**IMPORT**

In [1]:
import pandas as pd
import json       
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [37]:
pd.options.display.max_columns=None

**Model Evaluation Metrics**

In [20]:
#def eval_metrics(actual, pred):
    #rmse = np.sqrt(mean_squared_error(actual, pred))
    #mae = mean_absolute_error(actual, pred)
    #r2 = r2_score(actual, pred)
    #return rmse, mae, r2

**GET THE DATAFRAME**

**df 92 columns (eleminated)(dummied)**

In [34]:
df=pd.read_pickle("golden_data_reduced_16june.pkl")
X=df.drop(['price'], axis=1).values
y=df['price'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
df.shape

(14990, 92)

In [6]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error

**1.RF (92 columns)**

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor(n_estimators=100, random_state=10)
rf_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=10, verbose=0, warm_start=False)

In [11]:
print(rf_reg.score(X_test,y_test))

0.955957083296823


In [12]:
y_pred = rf_reg.predict(X_test)
mean_absolute_error(y_test, y_pred)

863.6365607624973

In [13]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

1491.5186450421684


**cross validation**

In [ ]:
from sklearn.model_selection import cross_val_score
accuraries = cross_val_score(estimator=rf_reg, X=X_train, y=y_train, cv=10)

In [332]:
accuraries.mean()

0.9527053689703878

**XGBOOST**

In [36]:
import xgboost
model = xgboost.XGBRegressor()
model.fit(X_train, y_train)

[10:38:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [37]:
model.score(X_test, y_test)

0.9340952852755392

In [38]:
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)

1217.1136413533502

In [39]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

1824.5211115380748


**LGBM (92 columns)**

In [16]:
import lightgbm

In [48]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [49]:
params={'metric' : 'l1'}

In [50]:
model_lgbm = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's l1: 4951.81
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 4532.63
[3]	valid_0's l1: 4162.72
[4]	valid_0's l1: 3832.91
[5]	valid_0's l1: 3535.66
[6]	valid_0's l1: 3263.6
[7]	valid_0's l1: 3029.22
[8]	valid_0's l1: 2817.16
[9]	valid_0's l1: 2637.58
[10]	valid_0's l1: 2464.68
[11]	valid_0's l1: 2319.1
[12]	valid_0's l1: 2183.45
[13]	valid_0's l1: 2061.8
[14]	valid_0's l1: 1954.3
[15]	valid_0's l1: 1862.97
[16]	valid_0's l1: 1776.28
[17]	valid_0's l1: 1700.64
[18]	valid_0's l1: 1632.97
[19]	valid_0's l1: 1573.65
[20]	valid_0's l1: 1521.53
[21]	valid_0's l1: 1472.81
[22]	valid_0's l1: 1431.28
[23]	valid_0's l1: 1392.43
[24]	valid_0's l1: 1360.92
[25]	valid_0's l1: 1334.64
[26]	valid_0's l1: 1307.82
[27]	valid_0's l1: 1286.26
[28]	valid_0's l1: 1265.63
[29]	valid_0's l1: 1250.17
[30]	valid_0's l1: 1235.16
[31]	valid_0's l1: 1222.16
[32]	valid_0's l1: 1208.95
[33]	valid_0's l1: 1198.22
[34]	valid_0's l1: 1184.19
[35]	valid_0's l1: 1174.44
[36]	

In [51]:
y_pred = model_lgbm.predict(X_test)
r2_score(y_test, y_pred)

0.9569139718006007

In [52]:
mean_absolute_error(y_test, y_pred)

866.5634219366598

In [53]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

1475.2270986152487


**CATBOOST 92 columns without categorical features**

In [26]:
df=pd.read_pickle("golden_data_reduced_16june.pkl")
X=df.drop(['price'], axis=1).values
y=df['price'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [27]:
from catboost import CatBoostRegressor
clf = CatBoostRegressor()
clf.fit(X_train, y_train)

0:	learn: 18636.0805616	total: 147ms	remaining: 2m 26s
1:	learn: 18106.9806105	total: 205ms	remaining: 1m 42s
2:	learn: 17606.5804222	total: 260ms	remaining: 1m 26s
3:	learn: 17110.6555141	total: 322ms	remaining: 1m 20s
4:	learn: 16629.4357481	total: 391ms	remaining: 1m 17s
5:	learn: 16175.2384872	total: 461ms	remaining: 1m 16s
6:	learn: 15735.2873380	total: 529ms	remaining: 1m 14s
7:	learn: 15316.5703682	total: 594ms	remaining: 1m 13s
8:	learn: 14909.0001655	total: 664ms	remaining: 1m 13s
9:	learn: 14505.4669595	total: 739ms	remaining: 1m 13s
10:	learn: 14115.1198451	total: 816ms	remaining: 1m 13s
11:	learn: 13733.0113684	total: 877ms	remaining: 1m 12s
12:	learn: 13357.3274850	total: 936ms	remaining: 1m 11s
13:	learn: 12992.7806442	total: 995ms	remaining: 1m 10s
14:	learn: 12636.3308270	total: 1.05s	remaining: 1m 9s
15:	learn: 12295.1511563	total: 1.12s	remaining: 1m 8s
16:	learn: 11964.7949063	total: 1.18s	remaining: 1m 8s
17:	learn: 11656.2553338	total: 1.22s	remaining: 1m 6s
18:	le

In [29]:
y_pred = clf.predict(X_test)
r2_score(y_test, y_pred)

0.9425031436925589

In [30]:
y_pred = clf.predict(X_test)
mean_absolute_error(y_test, y_pred)

1134.6081694266322

In [31]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

1704.1691559042279


**CATBOOST (98 columns) with categorical features,no get_dummy**

In [35]:
df3=pd.read_pickle("golden_data_not_dummied.pkl")
df3=df3.drop([14994, 14993, 14992, 14991])
X3=df3.drop(['price'], axis=1)
y3=df3['price']
from sklearn.model_selection import train_test_split
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size = 0.2, random_state = 0)

In [38]:
X_train3.head()

,displacement,make,model,hp,km,ss_abs,ss_adaptive_cruise_control,ss_adaptive_headlights,ss_alarm_system,ss_blind_spot_monitor,ss_central_door_lock,ss_central_door_lock_with_remote_control,ss_daytime_running_lights,ss_driver_drowsiness_detection,ss_driver_side_airbag,ss_electronic_stability_control,ss_emergency_brake_assistant,ss_emergency_system,ss_fog_lights,ss_head_airbag,ss_immobilizer,ss_isofix,ss_led_daytime_running_lights,ss_led_headlights,ss_lane_departure_warning_system,ss_night_view_assist,ss_passenger_side_airbag,ss_power_steering,ss_rear_airbag,ss_side_airbag,ss_tire_pressure_monitoring_system,ss_traction_control,ss_traffic_sign_recognition,ss_xenon_headlights,consumption_comb,body,body_color,gearing_type,inspection_new,age,ent_media_bluetooth,ent_media_cd_player,ent_media_digital_radio,ent_media_hands_free_equipment,ent_media_mp3,ent_media_on_board_computer,ent_media_radio,ent_media_sound_system,ent_media_television,ent_media_usb,fuel,nr_of_doors,nr_of_seats,type,warranty,cc_air_conditioning,cc_air_suspension,cc_armrest,cc_automatic_climate_control,cc_auxiliary_heating,cc_cruise_control,cc_electric_starter,cc_electric_tailgate,cc_electrical_side_mirrors,cc_electrically_adjustable_seats,cc_electrically_heated_windshield,cc_heads_up_display,cc_heated_steering_wheel,cc_hill_holder,cc_keyless_central_door_lock,cc_leather_seats,cc_leather_steering_wheel,cc_light_sensor,cc_lumbar_support,cc_massage_seats,cc_multi_function_steering_wheel,cc_navigation_system,cc_panorama_roof,cc_park_distance_control,cc_parking_assist_system_camera,cc_parking_assist_system_self_steering,cc_parking_assist_system_sensors_front,cc_parking_assist_system_sensors_rear,cc_power_windows,cc_rain_sensor,cc_seat_heating,cc_seat_ventilation,cc_split_rear_seats,cc_start_stop_system,cc_sunroof,cc_tinted_windows,cc_wind_deflector,cc_windshield,ext_alloy_wheels,ext_touch_screen,ext_trailer_hitch,ext_voice_control
2258,999.0,Audi,A1,70.0,50,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.6,Compact,Blue,Manual,0,0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,gasoline,5.0,5.0,Pre-registered,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15680,1598.0,Renault,Espace,118.0,20,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,4.7,Van,White,Automatic,1,1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,diesel,5.0,7.0,Pre-registered,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
5992,998.0,Opel,Astra,77.0,37889,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,4.3,Station wagon,Black,Automatic,0,2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,gasoline,5.0,5.0,Used,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
6420,999.0,Opel,Astra,77.0,50,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,4.5,Sedans,Grey,Semi-automatic,0,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,gasoline,5.0,5.0,Employee's car,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
14554,898.0,Renault,Clio,66.0,20187,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,5.0,Sedans,White,Manual,0,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,gasoline,5.0,5.0,Used,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.

In [3]:
pd.options.display.max_columns=None

In [47]:
#from catboost import CatBoostRegressor
categorical_features_indices = np.where(X3.dtypes == np.object)[0]

In [48]:
categorical_features_indices

array([ 1,  2, 35, 36, 37, 50, 53], dtype=int64)

In [39]:
X_train3.columns.get_loc("age")

39

In [32]:
#or
#a=X_train3.columns[X_train3.dtypes == 'object']
#print(a)
#Index(['make', 'model', 'body', 'body_color', 'gearing_type', 'fuel', 'type'], dtype='object')

#or
#my_list = list(X_train3.select_dtypes(include=['object']).columns)
#print(my_list)
#['make', 'model', 'body', 'body_color', 'gearing_type', 'fuel', 'type']

#then:
#col_numbers=[]
#for i in my_list:
#    col_numbers.append(X_train3.columns.get_loc(i))
#print(col_number)
#categorical_features_indices = col_numbers

In [49]:
from catboost import CatBoostRegressor
clf = CatBoostRegressor()
clf.fit(X_train3, y_train3, cat_features=categorical_features_indices)

0:	learn: 18634.7811834	total: 37.3ms	remaining: 37.3s
1:	learn: 18111.9498658	total: 81ms	remaining: 40.4s
2:	learn: 17616.8686735	total: 120ms	remaining: 39.9s
3:	learn: 17127.6903164	total: 161ms	remaining: 40.2s
4:	learn: 16656.3167317	total: 202ms	remaining: 40.2s
5:	learn: 16221.6763552	total: 217ms	remaining: 36s
6:	learn: 15767.1649597	total: 262ms	remaining: 37.2s
7:	learn: 15331.9557200	total: 296ms	remaining: 36.7s
8:	learn: 14909.7951805	total: 340ms	remaining: 37.4s
9:	learn: 14503.5415030	total: 374ms	remaining: 37s
10:	learn: 14115.6893598	total: 417ms	remaining: 37.5s
11:	learn: 13743.1466641	total: 456ms	remaining: 37.5s
12:	learn: 13374.2275895	total: 500ms	remaining: 38s
13:	learn: 13014.7150426	total: 525ms	remaining: 37s
14:	learn: 12658.0093017	total: 567ms	remaining: 37.2s
15:	learn: 12315.4997199	total: 601ms	remaining: 36.9s
16:	learn: 11981.4851879	total: 637ms	remaining: 36.9s
17:	learn: 11663.5377056	total: 676ms	remaining: 36.9s
18:	learn: 11361.2672180	tot

In [50]:
y_pred = clf.predict(X_test3)
r2_score(y_test3, y_pred)

0.9449063492727058

In [13]:
y_pred = clf.predict(X_test3)
mean_absolute_error(y_test3, y_pred)

1106.555940682657

In [14]:
rmse = np.sqrt(mean_squared_error(y_test3, y_pred))
print(rmse)

1665.9127792201368


Similar to CatBoost, LightGBM can also handle categorical features by taking the input of feature names. 
It does not convert to one-hot coding, and is much faster than one-hot coding.
LGBM uses a special algorithm to find the split value of categorical features 
Note: You should convert your categorical features to int type before you construct Dataset for LGBM. It does not accept string values even if you passes it through categorical_feature parameter.

**LGBM GridSearch**

In [15]:
df=pd.read_pickle("golden_data_reduced_16june.pkl")
X=df.drop(['price'], axis=1).values
y=df['price'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [17]:
lg = lgb.LGBMRegressor()

In [31]:
param_dist = {"max_depth": [5, 10, 15],
              "learning_rate" : [0.1, 0.2, 0.3],
              "num_leaves": [30, 100, 200],
              "n_estimators": [100]
             }

In [32]:
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3)
grid_search.fit(X_train,y_train)
grid_search.best_estimator_

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=15,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=100, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [12]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

In [33]:
params = {'metric' : 'l1', "max_depth": 15, "learning_rate" : 0.1, "num_leaves": 100}

In [34]:
model_lgbm = lgb.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's l1: 4922.58
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 4469.09
[3]	valid_0's l1: 4067.6
[4]	valid_0's l1: 3710.69
[5]	valid_0's l1: 3388.2
[6]	valid_0's l1: 3101.28
[7]	valid_0's l1: 2848.78
[8]	valid_0's l1: 2623.36
[9]	valid_0's l1: 2425.07
[10]	valid_0's l1: 2245.54
[11]	valid_0's l1: 2089.43
[12]	valid_0's l1: 1952.7
[13]	valid_0's l1: 1833.55
[14]	valid_0's l1: 1729.22
[15]	valid_0's l1: 1634.37
[16]	valid_0's l1: 1553.08
[17]	valid_0's l1: 1480.61
[18]	valid_0's l1: 1415.57
[19]	valid_0's l1: 1359.36
[20]	valid_0's l1: 1309.57
[21]	valid_0's l1: 1267.54
[22]	valid_0's l1: 1231.03
[23]	valid_0's l1: 1198.11
[24]	valid_0's l1: 1166.38
[25]	valid_0's l1: 1141.76
[26]	valid_0's l1: 1119.6
[27]	valid_0's l1: 1100.54
[28]	valid_0's l1: 1083.16
[29]	valid_0's l1: 1070.32
[30]	valid_0's l1: 1058.45
[31]	valid_0's l1: 1047.57
[32]	valid_0's l1: 1040.41
[33]	valid_0's l1: 1033.59
[34]	valid_0's l1: 1026.08
[35]	valid_0's l1: 1019.36
[36]	

In [35]:
y_pred = model_lgbm.predict(X_test)
r2_score(y_test, y_pred)

0.9550245165858309